In [1]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1647448853284_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Installing pandas package

In [2]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1
boto                       2.49.0
jmespath                   0.9.4
lxml                       4.4.1
mysqlclient                1.4.4
nltk                       3.4.5
nose                       1.3.4
numpy                      1.14.5
pip                        21.3.1
py-dateutil                2.2
python36-sagemaker-pyspark 1.2.6
pytz                       2019.3
PyYAML                     3.11
setuptools                 59.6.0
six                        1.12.0
soupsieve                  1.9.4
wheel                      0.37.1
windmill                   1.6

In [3]:
sc.install_pypi_package("pandas==1.0.5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Package                    Version
-------------------------- -------
beautifulsoup4             4.8.1
boto                       2.49.0
jmespath                   0.9.4
lxml                       4.4.1
mysqlclient                1.4.4
nltk                       3.4.5
nose                       1.3.4
numpy                      1.14.5
pandas                     1.0.5
pip                        21.3.1
py-dateutil                2.2
python-dateutil            2.8.2
python36-sagemaker-pyspark 1.2.6
pytz                       2019.3
PyYAML                     3.11
setuptools                 59.6.0
six                        1.12.0
soupsieve                  1.9.4
wheel                      0.37.1
windmill                   1.6

## Get input and output folder in S3

In [35]:
songPath = 's3a://udacity-dend/song_data/*/*/*/*.json'
logPath = 's3a://udacity-dend/log_data/*/*/*.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Make sure to add a valid output S3 path, otherwise you'll get an error while writing to s3**

In [36]:
output = 's3a://storage-for-sparkify/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read songs data

In [37]:
df_song = spark.read.json(songPath)
df_song.persist()
df_song.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

### Check schema
By default schema is inferred when importing data from json files

In [38]:
df_song.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)

## Songs table

In [39]:
song_cols = ['song_id', 'title', 'artist_id', 'year', 'duration']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- First required columns are selected.
- Duplicated rows are dropped.
- Blank spaces ou empty strings for `song_id` column are replaced by None (null).
- Row with null values in `song_id` column are dropped.

In [40]:
songs_table = df_song.select(song_cols)\
                     .distinct()\
                     .withColumn('song_id', F.when(F.col('song_id') == '', None).otherwise(F.col('song_id')))\
                     .na.drop(subset=['song_id'])
songs_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14896

In [41]:
songs_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOOORVX12A6D4FA612|Gimme A Chance  (...|ARKDZJP1187B98DD0C|2002|230.89587|
|SOQWBCS12A58A7C2C3|         Train Track|ARV62991187B99DE55|2007|168.64608|
|SOOQCOC12A8C132A5A|Heavy On My Mind ...|AR531IG1187FB59B35|2007|319.84281|
|SOVEHRU12AB017CEE6|Tomorrow We Sail ...|AR2LRXK1187FB593DD|   0|324.85832|
|SOPOOMV12AF72AC204|     She Is Michelle|ARPONRM1187B99B272|   0|373.65506|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows

### Writing songs table to parquet files into S3

In [42]:
df_pandas = songs_table.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- The `year` column is skewed which make it not a good choice for partioning. **I have terminated the cluster after 45 minutes**, partioning by `year` makes writing process very slow.
- Partition by number of Spark workers: `df.repartition(number_of_workers)` is another solution but I had to terminate also the cluster. So I didn't see any improvements.
- So writing without any partition is much quicker in this case.

In [43]:
df_pandas.year.value_counts()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0       4762
2007     871
2006     791
2008     779
2005     762
2009     738
2004     613
2003     563
2002     440
2001     404
1999     374
2000     349
1997     290
1998     285
1995     259
1996     237
2010     234
1994     231
1992     195
1993     184
1991     140
1990     117
1989     107
1988     101
1987      87
1986      86
1982      73
1981      56
1983      56
1984      55
1979      53
1978      53
1980      52
1985      49
1977      47
1970      39
1975      38
1971      38
1968      36
1972      32
1974      31
1969      30
1976      27
1973      27
1967      21
1966      20
1964      10
1965       8
1963       8
1960       6
1957       6
1961       5
1959       5
1962       5
1956       3
1954       3
1958       2
1927       1
1952       1
1944       1
Name: year, dtype: int64

In [44]:
songs_table.write.parquet(output + 'songs_table', mode='Overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Artists table

In [45]:
artist_cols = ["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- First required columns are selected.
- Duplicated rows are dropped.
- Blank spaces ou empty strings for `artist_id` column are replaced by None (null).
- Row with null values in `artist_id` column are dropped.

In [46]:
artists_table = df_song.select(artist_cols)\
                       .distinct()\
                       .withColumn('artist_id', F.when(F.col('artist_id') == '', None).otherwise(F.col('artist_id')))\
                       .na.drop(subset=['artist_id'])
artists_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10025

In [47]:
artists_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+----------------+--------------------+---------------+----------------+
|         artist_id|     artist_name|     artist_location|artist_latitude|artist_longitude|
+------------------+----------------+--------------------+---------------+----------------+
|AR86HDW1187FB37331|McDonald & Giles|                    |           null|            null|
|ARX2F7M1187B9A5563|            Maná|  NY - New York City|       40.71455|       -74.00712|
|ARI66OL1187B9AFA5B|      Wilderness|       Baltimore, MD|       39.29055|        -76.6096|
|AROBHXA1187B993C7E|    The Unicorns|Montreal, Quebec,...|       45.51228|       -73.55439|
|AR059HI1187B9A14D7|      Roy Drusky|         Atlanta, GA|       33.74831|       -84.39111|
+------------------+----------------+--------------------+---------------+----------------+
only showing top 5 rows

### Writing artists table to parquet files into S3

In [48]:
artists_table.write.parquet(output + 'songs_table', mode='Overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Read log data

In [49]:
df_log = spark.read.json(logPath)
df_log.persist()
df_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

In [50]:
df_log.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

### Converting `userId` type from string to integer

In [51]:
df_log.withColumn('userId', F.when(F.col('userId') == '', None).otherwise(F.col('userId')))\
      .withColumn("userId", F.col("userId").cast(T.IntegerType()))
df_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8056

Filtering by `NextSong` page

In [52]:
df_log = df_log.filter(F.col("page") == "NextSong")
df_log.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6820

## Users table

In [53]:
user_cols = ["user_id", "first_name", "last_name", "gender", "level"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- First renaming `userId`, `firstName`, `lastName` columns.
- Required columns are selected.
- Duplicated rows are dropped.
- Row with null values in `user_id` column are dropped.

In [54]:
users_table = df_log.withColumnRenamed('userId', 'user_id')\
                    .withColumnRenamed('firstName', 'first_name')\
                    .withColumnRenamed('lastName', 'last_name')\
                    .select(user_cols)\
                    .distinct()\
                    .na.drop(subset=['user_id'])
users_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

104

In [55]:
users_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     26|      Ryan|    Smith|     M| free|
|      7|    Adelyn|   Jordan|     F| free|
|     71|    Ayleen|     Wise|     F| free|
|     81|    Sienna|    Colon|     F| free|
|     23|    Morris|  Gilmore|     M| free|
+-------+----------+---------+------+-----+
only showing top 5 rows

### Writing users table to parquet files into S3

In [56]:
users_table.write.parquet(output + 'users_table', mode='Overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Time table

### Converting `epoch` to timestamp

In [57]:
df_log = df_log.na.drop(subset=['ts'])\
               .withColumn('start_time', F.to_timestamp(F.col("ts") / 1000))
df_log.select('ts','start_time').show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-----------------------+
|ts           |start_time             |
+-------------+-----------------------+
|1542241826796|2018-11-15 00:30:26.796|
|1542242481796|2018-11-15 00:41:21.796|
|1542242741796|2018-11-15 00:45:41.796|
|1542253449796|2018-11-15 03:44:09.796|
|1542260935796|2018-11-15 05:48:55.796|
+-------------+-----------------------+
only showing top 5 rows

- Selecting `start_time` column.
- Extracting `hour`, `day`, `week`, `month`, `year` and `weekday` from `start_time`.

In [58]:
time_table = df_log.select('start_time')\
                   .withColumn('hour', F.hour(F.col("start_time")))\
                   .withColumn('day', F.dayofmonth(F.col("start_time")))\
                   .withColumn('week', F.weekofyear(F.col("start_time")))\
                   .withColumn('month', F.month(F.col("start_time")))\
                   .withColumn('year', F.year(F.col("start_time")))\
                   .withColumn('weekday', F.dayofweek(F.col("start_time")))
time_table.show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+----+---+----+-----+----+-------+
|start_time             |hour|day|week|month|year|weekday|
+-----------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:26.796|0   |15 |46  |11   |2018|5      |
|2018-11-15 00:41:21.796|0   |15 |46  |11   |2018|5      |
|2018-11-15 00:45:41.796|0   |15 |46  |11   |2018|5      |
|2018-11-15 03:44:09.796|3   |15 |46  |11   |2018|5      |
|2018-11-15 05:48:55.796|5   |15 |46  |11   |2018|5      |
+-----------------------+----+---+----+-----+----+-------+
only showing top 5 rows

In [59]:
time_table.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- start_time: timestamp (nullable = true)
 |-- hour: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- week: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- weekday: integer (nullable = true)

### Writing time table to parquet files into S3

In [60]:
time_table.write.parquet(output + 'time_table', partitionBy=['year', 'month'], mode='Overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Songplays table

In [61]:
songplays_cols = ['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- First renaming columns.
- Dropping rows with null values in `song` and `artist` columns (because I'll be joinning dataframes based on them)

In [62]:
df_songplays = df_log.withColumnRenamed('userId', 'user_id')\
                     .withColumnRenamed('sessionId', 'session_id')\
                     .withColumnRenamed('userAgent', 'user_agent')\
                     .na.drop(subset=['song','artist'])\
                     .select('start_time', 'user_id', 'level', 'song', 'artist', 'session_id', 'location', 'user_agent')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

- First joinning on `song`=`title` and `artist`=`artist_name`.
- Selecting required columns.

In [63]:
songplays_table = df_song.join(df_songplays, (df_song.artist_name == df_songplays.artist) & 
                                             (df_song.title == df_songplays.song))\
                         .withColumn("songplay_id", F.row_number().over(Window.partitionBy().orderBy(F.col('session_id'))))\
                         .select(songplays_cols)
songplays_table.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

333

In [64]:
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|          1|2018-11-05 15:31:...|     24| paid|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|        23|Lake Havasu City-...|"Mozilla/5.0 (Win...|
|          2|2018-11-03 01:12:...|     53| free|SOSELMV12A6D4FCF5A|ARWD25M1187FB4C563|        52|   Klamath Falls, OR|"Mozilla/5.0 (Win...|
|          3|2018-11-05 05:57:...|     57| free|SOCGOZK12A8151BD5D|ARM0P6Z1187FB4D466|        56|San Antonio-New B...|"Mozilla/5.0 (Mac...|
|          4|2018-11-02 18:36:...|     71| free|SOBBZPM12AB017DF4B|ARH6W4X1187B99274F|        70|        Columbia, SC|"Mozilla/5.0 (iPh...|
|          5|2018-11

### Writing songplays table to parquet files into S3
Adding `month` and `year` columns in order to partition by `year` and `month`

In [65]:
songplays_table = songplays_table.withColumn('month', F.month(F.col("start_time")))\
                                 .withColumn('year', F.year(F.col("start_time")))
songplays_table.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----+----+
|songplay_id|          start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|month|year|
+-----------+--------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+-----+----+
|          1|2018-11-05 15:31:...|     24| paid|SOBONKR12A58A7A7E0|AR5E44Z1187B9A1D74|        23|Lake Havasu City-...|"Mozilla/5.0 (Win...|   11|2018|
|          2|2018-11-03 01:12:...|     53| free|SOSELMV12A6D4FCF5A|ARWD25M1187FB4C563|        52|   Klamath Falls, OR|"Mozilla/5.0 (Win...|   11|2018|
|          3|2018-11-05 05:57:...|     57| free|SOCGOZK12A8151BD5D|ARM0P6Z1187FB4D466|        56|San Antonio-New B...|"Mozilla/5.0 (Mac...|   11|2018|
|          4|2018-11-02 18:36:...|     71| free|SOBBZPM12AB017DF4B|ARH6W4X1187B99274F|        

In [66]:
songplays_table.write.parquet(output + 'songplays_table', partitionBy=['year', 'month'], mode='Overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…